In [3]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from tfidf_featurizer import combine_features, tfidf_featurize
from training import train, do_ensemble, do_train
from training_traditional import train_and_evaluate
from eval_erisk import evaluate, ensemble_vote
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow
import numpy as np

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


### With max_size = 10 and new data, sample_weights=10, oversample False, include_new_data=True

In [5]:
y_preds = {}

In [6]:
# prepare training data

In [7]:
train_users, y_train, test_users, y_test, train_samples, X_train, X_test = windowfy(window_size=10, max_size=20, sample_weights_size=20, is_oversample=True, include_new_data=False)
feats_train, feats_test = featurize(calculate_feats=True, include_feats=["first_prons","sentiment","nssi"], 
                       train_users=train_users, test_users=test_users, discretize=True, normalize=True)
tfidf_train, tfidf_test = tfidf_featurize(train_users, test_users, max_features=50000)

feats_train_comb, feats_test_comb = combine_features([tfidf_train, feats_train], [tfidf_test, feats_test])

feats_train_comb = feats_train_comb.toarray()
feats_test_comb = feats_test_comb.toarray()

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%
Oversampling train users
After oversample: positive messages: 2792, negative messages: 3491
Data size: 6283

Finished windowfying
Featurizing calculate_feats=True, normalize=True, discretize=True, discretize_size=10, include_feats=['first_prons', 'sentiment', 'nssi']
Initialized numpy random and tensorflow random seed at 42
Data size: 6283, 6283
Data size: 4650, 4650
Calculating first prons
Calculating sentiment
Calculating NSSI words
Calculating first prons
Calculating sentiment
Calculating NSSI words
Normalizing features
Discretizing


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:188: UserWarning: Feature 5 is constant and will be replaced with 0.
  "replaced with 0." % jj)


Is the combined the same from tfidf: False


In [8]:
# traditional classifiers

In [9]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="svm", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["svm"] = y_pred

              precision    recall  f1-score   support

           0       0.83      0.98      0.90      3602
           1       0.84      0.28      0.42      1048

    accuracy                           0.83      4650
   macro avg       0.83      0.63      0.66      4650
weighted avg       0.83      0.83      0.79      4650

[[3546   56]
 [ 752  296]]
{'precision': 0.7361111111111112, 'recall': 0.5096153846153846, 'F1': 0.6022727272727273, 'ERDE_5': 0.25669864310161844, 'ERDE_50': 0.13161086240911302, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5787959924069085}


In [10]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="bayes", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["bayes"] = y_pred

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      3602
           1       0.58      0.48      0.53      1048

    accuracy                           0.81      4650
   macro avg       0.72      0.69      0.70      4650
weighted avg       0.79      0.81      0.80      4650

[[3237  365]
 [ 541  507]]
{'precision': 0.49324324324324326, 'recall': 0.7019230769230769, 'F1': 0.5793650793650794, 'ERDE_5': 0.28911563836223, 'ERDE_50': 0.11687876196703907, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5567812900901438}


In [11]:
# deep learning classifiers
batch_size=32
model_name = "cnn_model"
iterations = 10

In [12]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=32
Generating embeddings
Data size: 6283
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
Evaluating
146/146 [==============================] - 4s 14ms/step - loss: 0.4729 - tp: 214.0000 - fp: 43.0000 - tn: 3559.0000 - fn: 834.0000 - accuracy: 0.8114 - precision: 0.8327 - recall: 0.2042 - f1_metric: 0.1659
Test Score: 0.47294318675994873
Test Accuracy: 214.0
146/146 [==============================] - 2s 12ms/step
Entered here
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      3602
           1       0.83      0.20      0.33      1048

    accuracy                           0.81      4650
   macro avg       0.82      0.60      0.61      4650
weighted avg       0.82      0.81      0.76      4650

[[3559   43]
 [ 834  214]]
Finished training and evaluation
{'precision': 0.7288135593220338, 'recall': 0.41346

{'precision': 0.7457627118644068, 'recall': 0.4230769230769231, 'F1': 0.5398773006134969, 'ERDE_5': 0.25442383822799597, 'ERDE_50': 0.15056251362272557, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5188327543927652}
Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=32
Generating embeddings
Data size: 6283
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
Evaluating
146/146 [==============================] - 2s 9ms/step - loss: 0.5139 - tp: 229.0000 - fp: 56.0000 - tn: 3546.0000 - fn: 819.0000 - accuracy: 0.8118 - precision: 0.8035 - recall: 0.2185 - f1_metric: 0.1886
Test Score: 0.5139224529266357
Test Accuracy: 229.0
146/146 [==============================] - 1s 8ms/step
Entered here
              precision    recall  f1-score   support

           0       0.81      0.98      0.89      3602
           1       0.80      0.22   

In [13]:
# deep learning classifiers
batch_size=32
model_name = "lstm_model_32"
iterations = 1

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

if len(model_resuls.keys() > 0):
    y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=lstm_model_32 and maxlen=1000 and batch size=32
Generating embeddings
Data size: 6283
Training with callback


In [ ]:
print(y_preds["cnn_model"].flatten())

In [ ]:
# ensemble

In [1]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["bayes"], y_preds["cnn_model"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

NameError: name 'ensemble_vote' is not defined

In [ ]:
evaluate(2, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

In [ ]:
evaluate(3, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)